**Table of contents**<a id='toc0_'></a>    
- [Feature selection](#toc1_1_)    
    - [Principle Component Analysis](#toc1_1_1_)    
    - [Feature importance from tree-based models (e.g., Random Forest)](#toc1_1_2_)    
- [model](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import sys
import os

import warnings
warnings.filterwarnings("ignore")
sys.path.append('/Users/jianbinlin/Documents/Master-Data-Spreadsheet/script/')

import utility
from utility import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(HTML("<style>.container { width:50% !important; }</style>"))
#display(HTML('<style>.output { max-width:800px !important; }</style>')) # control output width
display(HTML("<style>.container { width:130% !important; }</style>"))

pd.set_option('display.max_columns', None)   # Show number of columns, None means show all columns
pd.set_option('display.max_rows', 50)        # show top and bottom 15, middle with ......
pd.set_option('display.width', None)         # Set the width of the display
pd.set_option('display.max_colwidth', None)  # Show full content of each column

24/11/27 15:46:15 WARN Utils: Your hostname, Jians-iMac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.23 instead (on interface en1)
24/11/27 15:46:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/27 15:46:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
loyalty_merchant = pd.read_csv('https://raw.githubusercontent.com/KevinJianLin/loyalty_program/refs/heads/main/Data/merchants.csv')
data_target = loyalty_merchant.copy()
df_profiling_loyalty_merchant = data_profiling(data_target)
df_profiling_loyalty_merchant()

,count,mean,std,min,25%,50%,75%,max,number of nan and none values,min_char,max_char,non-ascii character,duplicated_rows,duplicated_columnss,shape and size,Completeness_Empty,Completeness_Space,float_col,float_col_legth,int_col,int_col_length
merchant_group_id,334696.0,3.102874e+04,31623.043426,1.000000,3612.000000,19900.000000,51707.250000,1.125860e+05,0,1,6,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
merchant_category_id,334696.0,4.231317e+02,252.898046,-1.000000,222.000000,373.000000,683.000000,8.910000e+02,0,1,3,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
subsector_id,334696.0,2.511640e+01,9.807371,-1.000000,19.000000,27.000000,33.000000,4.100000e+01,0,1,2,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
numerical_1,334696.0,1.147638e-02,1.098154,-0.057471,-0.057471,-0.057471,-0.047556,1.837351e+02,0,8,12,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
numerical_2,334696.0,8.103110e-03,1.070497,-0.057471,-0.057471,-0.057471,-0.047556,1.820793e+02,0,8,12,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
category_1,334696.0,2.103103e-02,0.143488,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0,1,1,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
most_recent_sales_range,334696.0,3.388233e+00,0.753297,0.000000,3.000000,4.000000,4.000000,4.000000e+00,0,1,1,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[numerical_1, numerical_2, avg_sales_lag3, avg_purchases_lag3, avg_sales_lag6, avg_purchases_lag6, avg_sales_lag12, avg_purchases_lag12, category_2]",9,"[merchant_group_id, merchant_category_id, subsector_id, active_months_lag3, active_months_lag6, active_months_lag12, city_id, state_id]",8
most_recent_purchases_range,334696.0,3.382565e+00,0.752935,0.000000,3.000000,4.000000,4.000000,4.000000e+00,0,1,1,0,0,0,"[(334696, 22), 140.44 Mb]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
data_target.head(5)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [6]:
data_target['subscribed']

0         no
1         no
2         no
3        yes
4         no
        ... 
31642     no
31643    yes
31644     no
31645     no
31646    yes
Name: subscribed, Length: 31647, dtype: object

In [18]:
label_encoder = LabelEncoder()
data_target['subscribed'] = label_encoder.fit_transform(data_target['subscribed'])


In [13]:
df_profiling_term_deposit.float_column

[]

In [14]:
df_profiling_term_deposit.int_column

['ID', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [15]:
df_profiling_term_deposit.cat_col

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'previous',
 'poutcome',
 'subscribed']

In [16]:
df_profiling_term_deposit.rest_columns

[]

In [ ]:
# Prepare the feature matrix and target
X = data_target[df_profiling_term_deposit.float_column + df_profiling_term_deposit.int_column + df_profiling_term_deposit.cat_col].drop(columns =['subscribed'])

y = data_target['subscribed']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
y

0         no
1         no
2         no
3        yes
4         no
        ... 
31642     no
31643    yes
31644     no
31645     no
31646    yes
Name: subscribed, Length: 31647, dtype: object

In [11]:
X_train.head(5)

,ID,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,day,month,previous,poutcome
18361,7670,44,5282,30,274,1,-1,0,management,married,tertiary,no,yes,no,unknown,30,may,0,unknown
2232,30787,54,184,6,13,6,-1,0,management,married,tertiary,no,no,no,cellular,6,feb,0,unknown
24219,33944,29,314,30,465,1,-1,0,student,single,secondary,no,no,no,cellular,30,apr,0,unknown
15644,27417,32,3816,21,53,3,-1,0,admin.,single,tertiary,no,yes,no,cellular,21,nov,0,unknown
8354,23064,38,0,26,101,4,-1,0,management,married,tertiary,no,no,no,cellular,26,aug,0,unknown


In [12]:
y_train.head(5)

18361     no
2232      no
24219    yes
15644     no
8354      no
Name: subscribed, dtype: object

## <a id='toc1_1_'></a>[Feature selection](#toc0_)

### <a id='toc1_1_1_'></a>[Principle Component Analysis](#toc0_)

### <a id='toc1_1_2_'></a>[Feature importance from tree-based models (e.g., Random Forest)](#toc0_)
Permutation importance
SHAP (SHapley Additive exPlanations) values
Recursive feature elimination
Lasso regularization

### 

# <a id='toc2_'></a>[model](#toc0_)

In [25]:

data_preprocessor_1 = ColumnTransformer(transformers = [
                                   #         ('text', TfidfVectorizer(max_features=500), text_col),
                                            ('float_col',StandardScaler(),df_profiling_term_deposit.int_column),
                                        #    ('cat_col', OneHotEncoder(), cat_col),
                                            ]) 

In [26]:
label_encoder.classes_

array(['no', 'yes'], dtype=object)

In [27]:
best_model ={}

In [28]:
# Loop through models and apply GridSearchCV
for model_name, model_infor in model_parameters_classification.items():


    # Create a pipeline for each model
    pipeline = Pipeline([('preprocessor',data_preprocessor_1),
                         ('classifier',model_infor['model'])])
    
    # Initialize GridSearchCV with model and parameter grid
    grid_search = GridSearchCV(estimator=pipeline,
                                param_grid = model_infor['params'],
                                cv = 5, # 5 fold cv
                                scoring = 'accuracy', # f1 etc
                                verbose =0, # not to display output
                                n_jobs = -1 # use all of cores
                                )
    grid_search.fit(X_train, y_train)

    best_model[model_name] = grid_search.best_estimator_
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")

    y_pred = grid_search.predict(X_test)
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(y_test, y_pred, target_names = label_encoder.classes_))

    # print(model_infor['model'])

ValueError: 
All the 180 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "/Users/jianbinlin/Documents/venv/lib/python3.10/site-packages/sklearn/pipeline.py", line 408, in _fit
    X, fitted_transformer = fit_trans

In [36]:
y_test

2308     0
22404    0
23397    0
25058    0
2664     1
        ..
2210     0
14144    1
23108    0
25703    0
29171    0
Name: default.payment.next.month, Length: 6000, dtype: int64

In [37]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])